In [2]:
#ML Imports
import gymnasium as gym
from stable_baselines3 import A2C
import pygame


In [2]:
env = gym.make("CartPole-v1", render_mode="rgb_array")

model = A2C("MlpPolicy",env,verbose=0)
model.learn(total_timesteps=10000)

vec_env = model.get_env()
obs = vec_env.reset()
for i in range(1):
    action, _state = model.predict(obs, deterministic = True)
    obs, reward, done, info = vec_env.step(action)
    vec_env.render(mode="human")
pygame.display.quit()
pygame.quit()
vec_env.close()


In [4]:
#Add in render_mode = "human" to have PyGame display the environment as it progresses
env = gym.make('Taxi-v3',render_mode='human')
env.reset()
env.render()